In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *

qm = QHJMall(MALL_KEY)

In [2]:
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

### 获取订单号的参数

```python

qm = QHJMall(MALL_KEY)
qm.get_order(order_type,**kargs)


# 请求类型 0 表示所有订单 1 未发货  2 已发货 3 已完成 4 申请退款 5 退款成功  10 未支付
"order_status": kargs.get("order_status", 0),
"order_time": kargs.get("order_time", ""), # today yesterday week month
"order_id": kargs.get("order_id", ""), # 订单号
"order_type": order_type,  # original free
"page": kargs.get("page", 1),  #

```


In [3]:
sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})


In [4]:
def get_orders(order_type,**kargs):
    data_l = []
    doc = pq(qm.get_order(order_type,**kargs))
    
    #   获取列明
    columns = []
    for th in doc('thead th').items():
        columns.append(th.text())

    for tr in doc('tbody tr').items():
        row_l = []
        for v in tr('td').items():
            row_l.append(v.text())
        data_l.append(row_l)
    item = doc('.pagination>li>a')
    if item.length >0:
        for page in range(2,int(item[-2].text)+1):
            kargs.update({"page":page})
            print(kargs)
            doc = pq(qm.get_order(order_type,**kargs))
            for tr in doc('tbody tr').items():
                row_l = []
                for v in tr('td').items():
                    row_l.append(v.text())
            data_l.append(row_l)

    data = pd.DataFrame(data_l,columns=columns)
    return data

In [5]:

def get_original_orders(**kargs):
    data = get_orders('original',**kargs)

    data_info = data['用户信息'].str.extract("""
    (?<=下单人：)(?P<收件人>[^\n]+)
    \s+手机：(?P<手机号码>[^\n]+)
    \s+收货地址：(?P<收货地址>[^\n]+)""",flags=re.S|re.X)

    data_dd = data['订单详情'].str.extract("""
        (?P<商品名>.+?)(?=（规格[：:])（
        规格[：:](?P<规格>.+?)(?=[，,]商品ID[：:])
        [，,]商品ID[：:](?P<商品ID>.+)）""",flags=re.S|re.X)
    data_r = pd.concat([data,data_info,data_dd],axis=1)
    
    return data_r

data_r = get_original_orders(order_status=1)
original_data = pd.merge(data_r,sku_goods[['商品ID','goods_type','单位','单价']] ,how='left',on='商品ID')

请求 {'order_status': 1, 'order_time': '', 'order_id': '', 'order_type': 'original', 'page': 1}


In [6]:
def get_free_order(**kargs):


    data = get_orders('free',**kargs)
    data = data.replace("三金美肤面膜 缓解过敏 修复护理|三金美肤面膜 维稳修护 强化屏障","三金护肤面膜 维稳修护 强化屏障",regex=True)
    patt = '三金补水面膜 镇静维稳 深层补水\\(补水\\)|三金美肤面膜 美白功效 健康肤色\\(美肤\\)|三金护肤面膜 维稳修护 强化屏障\\(护肤\\)|三金水光针\\(水光针\\)'
    data['商品名'] = data['订单详情'].str.extract('(?P<商品名>%s)'%patt,expand=False).str.strip()
    
    data_info = data['用户信息'].str.extract("""
        (?<=下单人：)(?P<收件人>[^\n]+)
        \s+手机：(?P<手机号码>[^\n]+)
        \s+收货地址：(?P<收货地址>[^\n]+)""",flags=re.S|re.X)
    data = pd.concat([data,data_info],axis=1)
    return data
data = get_free_order(order_status=1)

free_goods = sku_goods[sku_goods['goods_type']=='free']
# patt= "|".join(free_goods['商品名'].unique()).replace("(","\(").replace(")","\)")
free_data = pd.merge(data
             ,free_goods,on='商品名',how='left')





请求 {'order_status': 1, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 1}
{'order_status': 1, 'page': 2}
请求 {'order_status': 1, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 2}


In [7]:
data = pd.concat([free_data,original_data],sort = True)
data['数量'] = data['数量'].astype(int)

data['下单时间'] = pd.to_datetime(data['下单时间'])
data['支付时间'] = pd.to_datetime(data['支付时间'])
data = data.replace({"支付金额":{"￥":""}},regex=True).fillna({"支付金额":0})
data['支付金额'] = data['支付金额'].astype(int)

In [8]:
l = ['订单号', '订单状态','商品ID', '下单时间', '支付时间','支付金额',

 '商品名', '规格', '单位', '单价', '数量', '备注',
    
'收件人', '手机号码', '收货地址','goods_type']

data_r = data[l]

data_r.to_excel("test.xlsx",index=False)

In [9]:
data_r['订单号']

0    19121152975655
1    19121154991001
2    19121198995498
3    19121110153995
4    19121155561005
5    19121156101101
0    19121150100515
1    19121151981015
2    19121110210097
3    19121154485098
4    19121010251551
Name: 订单号, dtype: object

In [16]:
data_r.loc[:,'name'] = "ORDER:NEW:20"+data_r['订单号'].str[:2] +":" + data_r['订单号'].str[2:4] + ":" + data_r['订单号'].str[4:6]

In [11]:
import datetime
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        
        if isinstance(obj, datetime.datetime):  
            return obj.strftime('%Y-%m-%d %H:%M:%S')  
        else:
            return super(MyEncoder, self).default(obj)



In [13]:


def uprow(row):

    js = json.dumps(row.to_dict(),cls=MyEncoder,ensure_ascii=False)
    conn.hset(row['name'],row['订单号'],js)
data_r.apply(uprow,axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
0    None
1    None
2    None
3    None
4    None
dtype: object